In [1]:
!pip install pytorch-lightning
!pip install torch-summary

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 KB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 KB 12.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 KB 18.4 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
from pytorch_lightning import loggers as pl_loggers
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torchmetrics

Matplotlib is building the font cache; this may take a moment.


# Pytorch-Lightning : Training made Easier

Time : 4 hours

In the Tutorial session, we used PyTorch to train different models for Binary Classification. In the tutorial, few things were done :


*   We created a Training/Testing Loop and trained our models
*   We created a Trainer Class to gather all loops to perform the Training/Testing.


As you have seen, writing the training and testing loop can quickly be indigest. One can get easily lost.

Let us introduce you Pytorch Lightning

<img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/a88de56e65d2ea6bc203ce178a1cecbe9b50a0ac/68747470733a2f2f6769746875622e636f6d2f5079546f7263684c696768746e696e672f7079746f7263682d6c696768746e696e672f7261772f312e342e392f646f63732f736f757263652f5f7374617469632f696d616765732f6c6f676f2e706e67">


Pytorch lightning will handle a lot of things for you. It creates a Trainer which is a Code Management trick used by many companies (Meta, Google..) in order to get much more digest code.


More Information on : https://www.pytorchlightning.ai/

Goal of this lab :

* Use Pytorch Lightning for Training
* Learn to use Pytorch-Lightning
* Do classification on MNIST, CIFAR-10


# I - Classify Numbers using Lightning

In this part, we are interested in classifying digit images ranging from 0 to 9. 
We will use the Lightning framework for code management. What's interesting about Lightning is that you can plug in your Torch modules without any modification.

## a - LightningDataModule : MNIST

As you have seen in the Tutorial, you need to create your Dataset Class.

As a reminder :    
 The Dataset class returns one sample of your dataset at a time. The main methods of the Dataset class are 

*   __getitem__ : which fetched a sample at a given index
*   __len__ : which returns the len of the total dataset

The Dataset is loaded into a DataLoader. That Dataloader is then used to **fetch and send data as batches** for your Model.

You will see that using Lightning makes things clearer. LightningDataModule allows you to write cleaner Code and fit easily your data to your model.

You can always, use the basic Pytorch Dataloader in a separate code.

We are first going to work with the MNIST dataset. There is already a MNIST class provided in the Torchvision library, so we don't have to code the Dataset implementation ourself.
* Fill in the blanks

### Exploratory Data Analysis : Discovering the Data

Before tackling the classification task it is essential to explore the data we are working on, and understand its specifics.
Perform an Exploratory Data Analysis (EDA) on the MNIST Dataset :

1.   What type of Data do you have ? (Images, Texts, Sound..)
2.   How many Data do you have ? 
3.   What's in a sample (1 element of the Dataset)
4.   Is the Dataset umbalanced ?
5.   What's the shape of any input sample ?
6.   ....



In [ ]:
# Loading the Training Split of MNIST Dataset
dataset_train  = MNIST('', train=True, download=True)
dataset_test = MNIST('', train=False, download=True)

In [ ]:
# TODO : What's the length of the train and test split ?


60000

In [ ]:
# TODO : Retrieve one sample of the Dataset.
sample = dataset[...]

# TODO : What is in a sample ? Print the sample to understand
print()

(<PIL.Image.Image image mode=L size=28x28 at 0x7F42DD677990>, 5)


In [ ]:
# TODO : Plot the image in the sample. Does it correspond to the second element of the sample ?

plt.plot(...)

In [ ]:
# TODO : What's the shape of the input image.
shape = ...

### Lightning DataModule : Dataset and DataLoader Embedded 

Pytorch Lightning introduces a new way to define and organize our dataset via "data module". They neatly encompass our training, validation and testing datasets and provide their dataloaders as well.

Have a look at : https://pytorch.org/vision/stable/datasets.html

We are now going to define a `LightningDataModule` for the MNIST dataset. 

In [ ]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self):
        super().__init__()
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        self.data_dir = ''
        self.batch_size_train, self.batch_size_valid, self.batch_size_test = 32,32,32

    def prepare_data(self):
        # This method is used to download beforehand the dataset if needed.
        # TODO : Load the train and test dataset.
        MNIST(self.data_dir, train=..., download=...)
        

    def setup(self, stage):
        # We need to setup our module. We have 
        #  1. A training set that we will **fit** our model to
        #  2. A testing set used to **test** our models prediction.
        
        # The stage variable corresponds to those two steps : 
        # stage in {fit, test, None}

        # First stage is 'fit' (or None)
        if stage == "fit" or stage is None:
            # We create a validation split to watch the training.

            # TODO : Which dataset do we load for training ?
            mnist_dataset = MNIST(self.data_dir, train=..., transform=self.transform)
            train_size = int(0.8 * len(mnist_dataset))
            test_size = len(dataset_train) - train_size
            mnist_train, mnist_valid =  torch.utils.data.random_split(mnist_dataset, [train_size, test_size])
            
            # TODO : Load the datasets as attributes of the Module. Don't forget you validation split
            self.mnist_train, _ = 

        # Second stage is 'test' 
        if stage == "test" or stage is None:

            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)
            # Question : What additional set can we create ? Why ?

    def train_dataloader(self):
        # TODO : Now create your Training DataLoader
        return ...

    def val_dataloader(self):
        # TODO : Now create your Validation DataLoader
        return ...

    def test_dataloader(self):
        # TODO : Now create your Testing DataLoader
        return ...



## b - LightningModule :  MNIST Classifier 

Similarly to `LightningDataModule`, pytorch Lightning provides a `LightningModule` that gathers every functions needed for the training and testing of our pytorch model.
 
Design a model to perform Classification using the `LightningModule` class layout. Again, ask yourself the following questions: 
* What task is it ?
* What data do I have ?
* What learning rate should I use ?
* What could be my loss ? Why ?
* What non-linearity should I use ?
* How do I evaluate my model ? (TorchMetrics is your friend)

In [ ]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, output_shape):
        super(MNISTClassifier,self).__init__()
        # what is the output_shape of your model ?
        self.output_shape = output_shape
        self.save_hyperparameters()
        # TODO : Define your model here, be careful, your model will be an instance of the class. Watch  out for the input data.


    def forward(self,x):
        # TODO : What would be the forward steps of this classifier ? Return the output of our classifier given an input batch x.
        ...
        return x

    def configure_optimizers(self):
        # TODO : Choose your optimizer : https://pytorch.org/docs/stable/optim.html
        optimizer = ...
        return optimizer

    def training_step(self, batch, batch_idx):
        # TODO : Define your Training Step
        # This method is pretty much similar to what your did in the Tutorial to train your model.
        x,y = batch     
        ...
        loss = 
        acc = 
        # Don't remove the next line, you will understand why later
        self.log('train_acc', acc)
        self.log('train_loss', loss)
        return loss


    def validation_step(self, batch, batch_idx):
        # TODO : Define your Validation Step
        # What is the difference between the Training and the Validation Step ?
        x,y = batch
        ...
        loss = ...
        acc = ...
        # Don't remove the next line, you will understand why later
        self.log('val_acc', acc)
        self.log('val_loss', loss)
    
    def test_step(self, batch, batch_idx):
        # TODO : Define your Test Step
        # What is the difference between the Training, Validation and Test Step ?
        x,y = batch
        ...
        loss = ...
        self.acc = ... # We accumulate every accuracy
        # Don't remove the next line, you will understand why later
        self.log('test_loss', loss)
        self.log('test_acc', self.acc)

    def test_epoch_start(self):
        self.acc = 0

    def test_epoch_end(self):
        self.acc = 
        self.log('Final Accuracy', self.acc)

## c - Did you say Train ?

Let's train the model. 

We create our so called Trainer that will handle a lot of thing for us. Lightning trainer is full of interesting assets that helps you for your training. The lightning trainer is a much more evolved Trainer than the one in the Tutorial.

To get a glance of what Lightning Trainer can give :
https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html

It also easily lets us using TensorBoard. 

In [ ]:
tb_logger = pl_loggers.TensorBoardLogger("introduction to Lightning")

dm = MNISTDataModule()
model = MNISTClassifier(10)

trainer = pl.Trainer(gpus=-1,max_epochs=10,accelerator='dp',logger=tb_logger)
trainer.fit(model, dm)


Oh it's training ! Happy ? Easy ? Let's test the model

## d - Did you say Test ?

For testing, well it's pretty easy 

In [ ]:
trainer.test(model)

## e - TensorBoard

TensorBoard is a really useful tool. Indeed, it let's you register interesting values during training and plot them INTERACTIVELY. You might have seen a self.log line in the Validation and Training steps. 
The self.log saves the loss value into a TensorBoard readable file. We can also add images or other values using self.log

In fact, look at the checkpoint created by the training. You might see 3 files :
* Checkpoint
* event.out....
* hparam.yaml

Let's open tensorboard to see how the training was. Tensorboard is loadable using magic_python commands.
More info on TensorBoard : https://www.tensorflow.org/tensorboard/get_started

Another popular alternative to Tensorboard, also usable with pytorch lightning, is "Weight and Biases".

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir "/content/introduction to Lightning/default/version_0"

Pytorch Lightning can be used along PyTorch. We encourage you to use PyTorch Lightning during your Lab Sessions and Career as it simplifies a lot of things for you (MultiGPU, Learning Rate Decay...)

<img src='https://c.tenor.com/VyApQ-jWyV0AAAAC/happy-borat.gif'>



# II - Classify Objects using Lightning 

We will now turn to another classification task, this time object classification. We use the CIFAR-10 dataset which is made of 10 different classes of objects which we aim to distinguish. This part of the lab will be less restricted and more free. You now should have a sense of how to use the Lightning Framework. 
Be creative.


## a - Baseline : Creating your own Model

In this first section, we will create a Simple Model and perform all steps from part 1 with the needed changes.

*   **What's your final accuracy ?**


### i - DataModule

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

In [ ]:
# TODO : EDA 

In [ ]:
# TODO : Create your DataModule

### ii - Module

In [ ]:
# TODO : Create your Module

### iii - Train

In [ ]:
# TODO : Train

### iv - Test

In [ ]:
# TODO : Test


**Does your model perform well on the CIFAR Dataset ?**




## b - The OG Model : Finetuning a Model

If your model performed well on the CIFAR-10 Dataset, congrats. But let's achieve better results. For industrial works, we often pretrain a model on a large dataset (ImageNet or internal Dataset), and then fine-tune the model on the Dataset of interest.

* **What's the intuition behind fine-tuning ?**

### i - Importing a Pretrained Model

We will import a ConvNext model. Why ? It's said to be a really good backbone that competes with the Transformer models. Let's load the model. 
We are going to use TorchSummary to print what the size of the inputs and outputs are.

In [ ]:

import torchvision
model = torchvision.models.convnext_small(weights='DEFAULT')

# TODO : Using torchsummary, print a summary of the model
from torchsummary import summary
summary(model)


In [ ]:
# TODO : Using torchsummary, send an image of the same size as a sample of CIFAR-10
summary(model, ...) # ... = input shape as a tuple (C,H,W)


* **What is the output size of the model ?**
* **What will be the issue of using this model as is to perform classification on the CIFAR-10 Dataset ?**


In [ ]:
# TODO : According to your answer to the previous questions, perform the changes.
# You can access each layers using model.name_of_layer

### ii - DataModule

In [ ]:
# TODO : EDA 

# TODO : Create your DataModule

### iii - Module



In [ ]:
# TODO : Create your Module

# Careful : How should your learning rate be ?

### iv - Train

In [ ]:
# TODO : Train

### v - Test

In [ ]:
# TODO : Test


* **What is your final accuracy ?**
* **Is Fine Tuning a model better than creating your own model ?**